In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import bambi as bmb
import seaborn as sns
from pygam.datasets import mcycle
from pygam import LinearGAM, s, f, GAM, l, utils
import numpy as np
import arviz as az
import pymc as pm


random_seed = 100

import warnings

warnings.simplefilter('ignore')

# GAMs and GPs: Flexibility and Calibration

Flexible Spline models risk overfit and need to be carefully calibrated against real data.

::: {.callout-note}
## Summary 
Statistical models both encode structure in their definition and learn structure as they are calibrated against real data. There is a tension between how much structure we encode and how much we allow the model to discover. Too much discovery turns the model into a black-box and we lose some interpretability, too much encoded structure and we bias the discovery and risk losing sight of the data genersting process. 

Generalised Additive spline models and Gaussian process models are a tractable set of mathematical abstractions fit for arbitrary function approximation. We demonstrate their flexibility and continue to explore the interplay between such free function approximation and theoretically imposed constraints in the case of hierarchical spline modelling of insurance losses.
:::

## Theoretical Posits and Scientific Commitments

The scientific enterprise must abstract away details of reality to provide insight into the structure of reality. Statistical models of reality are cobbled together from a variety of abstractions; probability distributions and functional relations cojoined and calibrated against observed data. We are building on shifting foundations. Good abstractions aim at modelling the structures or _real patterns_ that matter.  

> [Yet] there is no way to establish fully secured, ... starting point of the sciences. There is no tabula rasa. We are like sailors who have to rebuild their ship on the open sea, without ever being able to dismantle it in dry-dock and reconstruct it from its best components. - Otto Neurath

Spline models are like the panes of wood we use to construct the ship and ride the next wave. In this blog post we're going to dive into modelling of non-linear functions and explore some of the tooling available in the python eco-system. We'll start by looking into Generalised Additive Models with splines in `pyGAM` before proceeding to look at Bayesian versions of spline modelling comparing the splines to Gaussian processes in `Bambi` and `PyMC`. Before showing how hierarchical bayesian models avoid some of the issues of overfit in simpler spline models. 

Our interest in these models stems from their flexibility to approximate functions of arbitrary complexity. We'll see how the methods work in the case of relatively straightforward toy example and then we'll apply each of the methods to deriving insights into the functional form of insurance loss curves. In this application we adapt a data set discussed in Mick Cooney's `Stan` [case study](https://mc-stan.org/users/documentation/case-studies/losscurves_casestudy.html) to demonstrate the power of hierarchical spline models. Throughout we'll draw on the discussion of these methods in Osvaldo Martin's _"Bayesian Analysis with Python"_ for practical details implementing these models. 

All of these methods need to be assessed with respect to their in-sample model fit and their out of sample performance. How can we best calibrate the model fits to perform reasonably well out of sample? How does the spline-constructs reflect an underlying reality, how can our posit's aid generalisation?

## Generalised Additive models

The canonical reference for GAMs is Simon Wood's _"Generalised Additive Models: An Introduction with R"_ which outlines in some detail the theoretical background of splines and univariate smoothers. The book stresses the trade-offs between the flexibility of splines and the need for cross-validation and penalised estimation methods for spline based modelling. 

In R these penalised models fits can be achieved in `mgcv` which incorporates a wilkinson like formula syntax for model specification: `y ~ s(x) + s(x1)`. 
The closest implementation in python is available in `PyGam` and we will adopt this package to illustrate the application of smoothing terms and the penalities. 

### PyGAM and Penalised Fits
In the next code block we load in an example data set on which to demonstrate univariate smoothing patterns using penalised splines. These models can be optimised by fitting differing strength penalities over a varying number of splines. Splines are used to construct non-linear functions of the input variable which are combined additively in our model equation. The penalities used in the optimisation routines constrain how "wiggly" these constructions will be. 

In [ ]:
X, y = mcycle(return_X_y=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(X, y)
ax.set_ylabel("Acceleration")
ax.set_xlabel("Time Step")
ax.set_title("Crash Test Dummy Acceleration \n Simulated Motorcycle Crash", fontsize=20)

Next we fit a number of different models to account for the herky-jerky nature of the data generating processs. We vary the parameterisations to see how the numbers of splines and strength of the penalty help account for the variation in $y$ over the support of $X$. 


In [ ]:
gam1 = LinearGAM(s(0, n_splines=5)).fit(X, y)
gam2 = LinearGAM(s(0, n_splines=7)).fit(X, y)
gam3 = LinearGAM(s(0, n_splines=10)).fit(X, y)
gam4 = LinearGAM(s(0, n_splines=15)).fit(X, y)
gam5 = LinearGAM(s(0, lam=.1)).fit(X, y)
gam6 = LinearGAM(s(0, lam=.5)).fit(X, y)
gam7 = LinearGAM(s(0, lam=5)).fit(X, y)
gam8 = LinearGAM(s(0, lam=15)).fit(X, y)


def plot_fit(gam, X, y, ax, t, c1='b', c2='r'):
    XX = gam.generate_X_grid(term=0, n=500)

    ax.plot(XX, gam.predict(XX), color=c2, linestyle='--')
    ax.plot(XX, gam.prediction_intervals(XX, width=.95), color=c1, ls='--')

    ax.scatter(X, y, facecolor='gray', edgecolors='none')
    ax.set_title(f"""95% prediction interval with {t} \n LL: {gam.statistics_['loglikelihood']}""");

fig, axs = plt.subplots(4,2, figsize=(10, 20))
axs = axs.flatten()
titles = ['5_splines', '7_splines', '10_splines', '15_splines',
'lam=.1', 'lam=.5', 'lam=5', 'lam=15']
gs = [gam1, gam2, gam3, gam4, gam5, gam6, gam7, gam8]
for ax, g, t in zip(axs, gs, titles):
    plot_fit(g, X, y, ax, t)


Here we've seen the `PyGAM` package applied to fitting our model to the data. In the formula specification we see `y ~ s(i)` where `i` denotes the index of the column variable in the X data. 

Over the range of of the x-axis we can see how the conditional expectation is more or less well fit to the data depending on how the penalities and complexity of the model is specified. 

## Optimising The Parameter Setting

We can see from the model summary what is going on under the hood. For a given model specification the summary will report a number of model-fit statistics such as the log-likelihood and the AIC. 


In [ ]:
## Naive Model manually specified splines
gam_raw = LinearGAM(s(0, n_splines=10), fit_intercept=False).fit(X, y)
print("log_likelihood:", gam_raw.statistics_['loglikelihood'])
print("AIC:", gam_raw.statistics_['AIC'])


The question then becomes what changes are induced in the model as we seek to optimise these model fit statistics. 


In [ ]:
## model optimised
gam = LinearGAM(s(0),  fit_intercept=False)
gam.gridsearch(X, y)
gam.summary()

Fortunately, this routine can be performed directly and results in the following differences between the naive and optimised model. 

### Plot the GAM fits


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_fit(gam_raw, X, y, ax, "Unoptimised Fit", c1='orange', c2='green')
plot_fit(gam, X, y, ax, "Optimised Fit")

This is all well and good. We've seen an approach to modelling that can capture eccentric patterns in raw data. But how does it work and why should we care?

If you're familiar with the nomenclature of machine learning, you should think of spline modelling as a variety of feature creation. It generates "synthetic" features over the range of the observed variable. These synthetic features are the splines in question. 

### Digression on the usage of "Splines" 

The history of the term "spline" is related to the history of draftmanship. Historically splines were thin strips of flexible wood or plastic that could be bent or shaped around a weight or "knot" points to express a traceable curve over the space of a "numberline". The elastic nature of the spline material allowed it to be bent around the knot points of curvature expressing a smooth or continuous bend. 

The mathematical technique apes these properties by defining a curve over in an analogous way. We specify "knots"  to carve up the support of the random variable $X$ into portions that require different weighting schemes to represent the outcome $y$ 

<img src="Spline.png" alt="spline" height="500"/>

### Extracting the Splines

We can extract the spline features used in the `PyGAM` by invoking the following commands to first identify the knot points and create the b-spline basis appropriate for the variable $X$. 


In [ ]:
knot_edges=utils.gen_edge_knots(X,dtype='numerical')
knots=np.linspace(knot_edges[0],knot_edges[-1],len(gam.coef_))

splines = utils.b_spline_basis(X, edge_knots=knot_edges, sparse=False)

splines_df = pd.DataFrame(splines, columns=[f'basis_{i}' for i in range(len(gam.coef_))])

splines_df.head(10)

These spline features range the extent of the of covariate space $X$ defining "partitions" of the space. The model "learns" to capture the shape of the outcome variable $y$ by figuring out how to weight the different portion of this spline basis matrix i.e. the linear combination of this basis matrix with the derived coefficients is a model of our outcome variable. 

Note how each row is 0 everywhere except within the columns that represent a partition of $X$. Additionally each row sum to unity. These properties are important because they ensure that any weighted combination of this basis represents the outcome variable in a controlled and quite granular manner. The more splines we use the more control we have of the representation.  

### Plotting the Weighted Spline

In [ ]:
ax = splines_df.dot(gam.coef_).plot(title='Weighted Splines', label='Weighted Combination of Spline Basis', figsize=(10, 6))
ax.set_ylabel("Acceleration")
ax.set_xlabel("Time Steps")
ax.legend();

In this manner we can see how the specification of a spline basis can help us model eccentric curves in an outcome space.They are like duct-tape tying down our rigging. Filling the gaps in our theory. If we hold this part down there, and jiggle another part here, our theory stays afloat. Spline constructions exemplify the contorted nature of empirically informed theory construction.

Next we'll see how to more directly work with the specification of basis splines, passing these feature matrices into Bambi models. 

## Bayesian Splines with bambi

Under the hood `Bambi` makes use of the ['formulae'](github.com/bambinos/formulae) package to allow for formula-like syntax to specify spline basis terms and mixed effect model terms. We leverage this to streamline the specification of spline models. 


In [ ]:
knots_6 = np.linspace(0, np.max(X), 6+2)[1:-1]
knots_10 = np.linspace(0, np.max(X), 10+2)[1:-1]
knots_15 = np.linspace(0, np.max(X), 15+2)[1:-1]

df = pd.DataFrame({'X': X.flatten(), 'y': y})
formula1 = 'bs(X, degree=0, knots=knots_6)'
formula2 = 'bs(X, degree=1, knots=knots_6, intercept=False)'
formula3 = 'bs(X, degree=3, knots=knots_6, intercept=False)'
formula4 = 'bs(X, degree=3, knots=knots_10, intercept=False)'
formula5 = 'bs(X, degree=3, knots=knots_15, intercept=False)'
model_spline1 = bmb.Model(f"y ~ {formula1}", df)
model_spline2 = bmb.Model(f"y ~ {formula2}", df)
model_spline3 = bmb.Model(f"y ~ {formula3}", df)
model_spline4 = bmb.Model(f"y ~ {formula4}", df)
model_spline5 = bmb.Model(f"y ~ {formula5}", df)
model_spline5

As you can see here we are specifying a range of different degrees of spline basis. The different degrees corresspond to the smoothness of the overlapping splines. 
The `degree=0` splines mean we specify a piecewise constant basis i.e. 0 or 1 within each region of the partition. But we can add more degrees to see more flexible representations of the space. 


In [ ]:
model_spline5.build()
model_spline5.graph()

This will become clearer as we plot the various spline basis matrices below. 

### Plot the Spline Basis

The below functions extract the basis specification from each model and plots the basis design for an increasingly complex series of spline basis matrices. 


In [ ]:
def plot_spline_basis(basis, X, ax, title="Spline Basis"):
    df = (
        pd.DataFrame(basis)
        .assign(X=X)
        .melt("X", var_name="basis_idx", value_name="y")
    )


    for idx in df.basis_idx.unique():
        d = df[df.basis_idx == idx]
        ax.plot(d["X"], d["y"])
    
    ax.set_title(title)
    return ax

def plot_knots(knots, ax):
    for knot in knots:
        ax.axvline(knot, color="0.1", alpha=0.4)
    return ax



fig, axs = plt.subplots(5, 1, figsize=(9, 20))
axs = axs.flatten()
axs.flatten()
B1 = model_spline1.response_component.design.common[formula1]
plot_spline_basis(B1, df["X"].values, ax=axs[0], title="Piecewise Constant Basis")
plot_knots(knots_6, axs[0]);

B2 = model_spline2.response_component.design.common[formula2]
ax = plot_spline_basis(B2, df["X"].values, axs[1], 
title="Piecewise Linear Basis")
plot_knots(knots_6, axs[1]);

B3 = model_spline3.response_component.design.common[formula3]
ax = plot_spline_basis(B3, df["X"].values, axs[2], 
title="Cubic Spline Basis (6 Knots)")
plot_knots(knots_6, axs[2]);

B4 = model_spline4.response_component.design.common[formula4]
ax = plot_spline_basis(B4, df["X"].values, axs[3], 
title="Cubic Spline Basis (10 Knots)")
plot_knots(knots_10, axs[3]);


B5 = model_spline5.response_component.design.common[formula5]
ax = plot_spline_basis(B5, df["X"].values, axs[4], 
title="Cubic Spline Basis (15 Knots)")
plot_knots(knots_15, axs[4]);


Now that we've seen the nature of the modelling splines we'll fit each model to the data and plot how the weighted splines matrices are able to represent the raw data. 

### Fit the Individual Spline Models

We now combine the spline components within a linear model fit, pulling and tying them together in such a way as to ape the shape of the observed sequence. 


In [ ]:
#| output: false

idata_spline1 = model_spline1.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True})

idata_spline2 = model_spline2.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True})

idata_spline3 = model_spline3.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True})

idata_spline4 = model_spline4.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True})

idata_spline5 = model_spline5.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True})

Plotting the fits

### Plot the Weighted Mean


In [ ]:
def plot_weighted_splines(B, idata, formula, ax, knots):
    posterior_stacked = az.extract(idata)
    wp = posterior_stacked[formula].mean("sample").values

    plot_spline_basis(B * wp.T, df["X"].values, ax)
    ax.plot(df.X.values, np.dot(B, wp.T), color="black", lw=3, label='Weighted Splines')
    plot_knots(knots, ax);
    ax.legend()



fig, axs = plt.subplots(5, 1, figsize=(10, 20))
axs = axs.flatten()
axs.flatten()

plot_weighted_splines(B1, idata_spline1, formula1, axs[0], knots_6)
plot_weighted_splines(B2, idata_spline2, formula2, axs[1], knots_6)
plot_weighted_splines(B3, idata_spline3, formula3, axs[2], knots_6)
plot_weighted_splines(B4, idata_spline4, formula4, axs[3], knots_10)
plot_weighted_splines(B5, idata_spline5, formula5, axs[4], knots_15)

Here we can see how the models with increasingly complex splines are more exactly able to fit the herky jerky trajectory of the outcome variable in each interval. The fewer the intervals, the less flexibility available to the model. 

### Compare Model Fits

As before we can evaluate these model fits and compare them based on leave-one-out cross validation scores and information theoretic complexity measures. 


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 7))
axs = axs.flatten()
models_dict = {"piecewise_constant": idata_spline1, "piecewise_linear": idata_spline2, "cubic_bspline": idata_spline3, "cubic_bspline_10": idata_spline4, 
"cubic_bspline_15": idata_spline5}
df_compare = az.compare(models_dict)
az.plot_compare(df_compare, ax=axs[0])
axs[0].get_legend().remove()
az.plot_compare(az.compare(models_dict, 'waic'), ax=axs[1])
axs[1].set_yticklabels([])
df_compare


Here we see that the extra complexity of using 15 splines leads to slightly worse performance measures than the less complex but seemingly adequate 10 splines.


In [ ]:
new_data = pd.DataFrame({"X": np.linspace(df.X.min(), df.X.max(), num=500)})
    
model_spline4.predict(idata_spline4, data=new_data, 
kind='pps', inplace=True)

idata_spline4

Next we plot the posterior predictive distribution of our observed variable and compare against the observed data. Additionally we plot the 89th and 50% HDI. 


In [ ]:
ax = az.plot_hdi(new_data['X'], idata_spline4['posterior_predictive']['y'], fill_kwargs={'alpha': 0.2, 'color':'firebrick'}, hdi_prob=0.89, figsize=(10, 8))

az.plot_hdi(new_data['X'], idata_spline4['posterior_predictive']['y'], fill_kwargs={'alpha': 0.8, 'color':'firebrick'}, hdi_prob=0.5)

y_mean = idata_spline4['posterior_predictive']['y'].mean(dim=('chain', 'draw'))

ax.plot(new_data['X'], y_mean, label='Expected posterior predictive', color='k')
ax.set_xlabel("Time Point")
ax.set_ylabel("Acceleration")

ax.scatter(df['X'], df['y'], label='Observed Datapoints')

ax.legend()

ax.set_title("Posterior Predictive Distribution \n Based on 10 Knots");

This represents a good a clean model fit to the observed data using univariate spline smoothers. Next we'll see another alternative approach to model this outcome variable using approximate gaussian processes. 

## Approximate Gaussian processes

The topic of gaussian processes is rich and complex. Too rich to be fairly covered in this blog post, so we'll just say that we're using a method designed for function approximation that makes use of drawing samples from a multivariate normal distribution under a range of different covariance relationships. We are in fact using the HSGP approximation of gaussian processes. 

Nevertheless the gaussian relationships can be somewhat intuitively interrogated by defining different combinations of covariance relationships with priors over the parameters governing the covariance of a sequence of points. For example consider the following parameterisations. 

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 10))
axs = axs.flatten()

def plot_cov_draws(ax1, ax2, lengthscale=3, sigma=13):
    cov = sigma**2 * pm.gp.cov.ExpQuad(1, lengthscale)
    X = np.linspace(0, 60, 200)[:, None]
    K = cov(X).eval()

    sns.heatmap(pd.DataFrame(K), center=0, xticklabels=[], yticklabels=[], ax=ax1, cmap='crest');
    ax1.set_title(f"Covariance Length Scale {lengthscale}")
    ax2.plot(
        X,
        pm.draw(
            pm.MvNormal.dist(mu=np.zeros(len(K)), cov=K, shape=K.shape[0]), draws=10, random_seed=random_seed
        ).T, color='blue', alpha=0.5
    )
    ax2.set_title(f"Samples from the GP prior \n lengthscale: {lengthscale}, sigma: {sigma}")
    plt.ylabel("y")
    plt.xlabel("X");

plot_cov_draws(axs[0], axs[1])
plot_cov_draws(axs[2], axs[3], lengthscale=10, sigma=13)

We've specified the range of X to reflect the support of the acceleration example and allowed the draws to be informed by a covariance function we have parameterised using the Exponentiated Quadratic kernel:

$$k(x, x') = \mathrm{exp}\left[ -\frac{(x - x')^2}{2 \ell^2} \right]$$

The plot on the left highlights the importance of the lengthscale parameter over the sequence. The wider the central shading is the more pronounced is the correlation among more points in the sequence. The patterns to the right show a good range of "wiggliness" that they should be flexible enough to capture the shape of the acceleration, if we can calibrate the model and derive posterior parameters against the observed data. 

### Priors on Gaussian Processes

Consider the following specification for the priors


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(9, 6))
axs = axs.flatten()
axs[0].hist(pm.draw(pm.InverseGamma.dist(mu=1, sigma=1), 1000), ec='black', bins=30);
axs[0].set_title("Priors for Lengthscale \n in ExpQuad Kernel")
axs[1].hist(pm.draw(pm.Exponential.dist(lam=1), 1000), ec='black', bins=30);
axs[1].set_title("Priors for Amplitude \n in ExpQuad Kernel")

We use these to specify priors on the Hilbert space approximation of gaussian priors available in the `Bambi` package. 


In [ ]:
prior_hsgp = {
    "sigma": bmb.Prior("Exponential", lam=1), # amplitude
    "ell": bmb.Prior("InverseGamma", mu=1, sigma=1) # lengthscale
}

# This is the dictionary we pass to Bambi
priors = {
    "hsgp(X, m=10, c=1)": prior_hsgp,
    "sigma": bmb.Prior("HalfNormal", sigma=4)
}
model_hsgp = bmb.Model("y ~ 0 + hsgp(X, m=10, c=1)", df, priors=priors)
model_hsgp


Here we've set the `m=10` to determine the number of basis vectors used in the Hilbert space approximation. The idea differs in detail from the spline based approximations we've seen, but it's perhaps useful to think of the process in the same vein. Here again we have a theory of the world expressed as a function of opaque components jerry-rigged for modelling some phenomena. 


In [ ]:
idata = model_hsgp.fit(inference_method="nuts_numpyro",target_accept=0.95, random_seed=121195, 
idata_kwargs={"log_likelihood": True})
print(idata.sample_stats["diverging"].sum().to_numpy())

This model fits and the sampling seems to have worked well. 


In [ ]:
az.plot_trace(idata, backend_kwargs={"layout": "constrained"}, figsize=(9, 15));


The lengthscale and sigma parameters we have learned by calibrating our priors against the data. The degree to which these parameters are meaningful depend a little on how familar you are with covariance matrix kernels and their properties, so we won't dwell on the point here. 


In [ ]:
az.summary(idata, var_names=['hsgp(X, m=10, c=1)_ell', 'hsgp(X, m=10, c=1)_sigma', 'y_sigma', 'hsgp(X, m=10, c=1)_weights'])


But again we can sample from the posterior predictive distribution of the outcome variable 


In [ ]:
model_hsgp.predict(idata, data=new_data, 
kind='pps', inplace=True)

idata

and plot the model fit to see if it can recover the observed data. 


In [ ]:
ax = az.plot_hdi(new_data['X'], idata['posterior_predictive']['y'], fill_kwargs={'alpha': 0.2, 'color':'firebrick'}, figsize=(9, 8))

az.plot_hdi(new_data['X'], idata['posterior_predictive']['y'], fill_kwargs={'alpha': 0.8, 'color':'firebrick'}, hdi_prob=0.5)

y_mean = idata['posterior_predictive']['y'].mean(dim=('chain', 'draw'))

ax.plot(new_data['X'], y_mean, label='Expected posterior predictive', color='k')

ax.scatter(df['X'], df['y'], label='Observed Datapoints')

ax.legend()

ax.set_title("Posterior Predictive Distribution \n Based on HSGP approximation")


And we can compare versus the spline models to see that by the aggregate performance measures our HSGP model seems to come out on top. 


In [ ]:
models_dict = {"piecewise_constant": idata_spline1, "piecewise_linear": idata_spline2, "cubic_bspline": idata_spline3, "cubic_bspline_10": idata_spline4, 
"cubic_bspline_15": idata_spline5, 'hsgp': idata}
df_compare = az.compare(models_dict)
df_compare


### Recap

So far we've seen how we can use splines and gauassian processes to model highly eccentric functional relationships where the function could be approximated with univariate smoothing routine. These are two distinct abstractions which seem adequately fit to the world, but demand very different ways of thinking about the underlying reality. This is no fundamental contradiction in so far as the world admits many descriptions. 

> [K]nowledge develops in a multiplicity of theories, each with it's limited utility ... These theories overlap very considerable in their logical laws and in much else, but that they add up to an integrated and consistent whole is only a worthy ideal and not a pre-requistite of scientific progress ... Let reconciliations proceed.  - W.V.O Quine in _Word and Object_

Another observation in a similar vein is that penalised spline models are provably [equivalent](https://www.tjmahr.com/random-effects-penalized-splines-same-thing/) to hierarchical regression (random effects) models. This is striking because the character of these types of model seems diametrically opposed. With spline models you jerry-rig your features to an optimisation goal, with hierarchical model you tend to impose theoretical structure to induce shrinkage. It's hard to see how this works - with a penalised spline model are you inducing a hierarchy of latent features you can't name? Should you even try to translate between the two!? The abstract components of our model is less graspable than the predictive performance. The qualitative character of theory buiding differs markedly across these abstractions and aesthetics and explainability feature heavily in model preference. 

Next we'll step up our abstractions and show how to use hierarchical modelling over spline fits to extract insight into the data generating process over a family of curves. In particular we'll focus on the development of insurance loss curves.

## Insurance Loss Curves: Hierarchical Spline Models

We draw on car insurance losses data set discussed in Mick Cooney's Stan case-study, but we simplify things for ourselves considerably by focusing on two types of loss and ensuring that each year under consideration has equal observations of the accruing losses. 


In [ ]:
loss_df = pd.read_csv('ppauto_pos.csv')
loss_df = loss_df[(loss_df['GRCODE'].isin([43, 353])) & (loss_df['DevelopmentYear'] < 1998)]

loss_df = loss_df[['GRCODE', 'AccidentYear', 'DevelopmentYear', 'DevelopmentLag', 'EarnedPremDIR_B', 'CumPaidLoss_B']]


loss_df.columns = ['grcode', 'acc_year', 'dev_year', 'dev_lag', 'premium', 'cum_loss']
loss_df['lr'] = loss_df['cum_loss'] / loss_df['premium']
loss_df = loss_df[(loss_df['acc_year'] <= 1992) & (loss_df['dev_lag'] <= 6)].reset_index(drop=True)

loss_df['year_code'] = loss_df['acc_year'].astype(str) + '_' + loss_df['grcode'].astype(str)
loss_df.sort_values(by=['year_code', 'acc_year', 'dev_lag'], inplace=True)
loss_df['standardised_premium'] = (loss_df.assign(mean_premium = np.mean(loss_df['premium']))
.assign(std_premium = np.std(loss_df['premium']))
.apply(lambda x: (x['mean_premium'] - x['premium']) /x['std_premium'], axis=1)
)

loss_df.head(12)

### Plot the Loss Curves
Here we have plotted the developing loss curves from two different coded insurance products. 


In [ ]:
pivot = loss_df.pivot(index=['dev_lag'], columns=['grcode', 'acc_year'], values='lr')
fig, axs = plt.subplots(1, 2, figsize=(9, 7))
pivot.plot(figsize=(10, 6), ax=axs[0])
axs[0].set_title("Loss Ratios by Year");
for c in pivot.columns:
    if 43 in c:
        color='red'
    else: 
        color='grey'
    axs[1].plot(pivot[c], color=color, label=c)
axs[1].legend()
axs[1].set_title("Loss Ratio by Group")


We want to model these curves collectively as instances of draws from a distribution of loss curves. To do so we will specify a `PyMC` hierarchical (mixed) spline model. To do so we will have a spline basis for the global hyper parameters `beta_g` and the individual parameters for each curve. Here we define a convenience function to generate the basis splines. 


In [ ]:
from patsy import bs as bs_patsy, dmatrix

def make_basis_splines(num_knots=3, max_dev=7):
    knot_list = np.linspace(0, max_dev, num_knots+2)[1:-1]
    dev_periods = np.arange(1, max_dev, 1)

    Bi = dmatrix(
        "bs_patsy(dev_periods, knots=knots, degree=3, include_intercept=True) - 1",
        {"dev_periods": dev_periods, "knots": knot_list},
    )

    Bg = dmatrix(
        "bs_patsy(dev_periods, knots=knots, degree=3, include_intercept=True) - 1",
        {"dev_periods": dev_periods, "knots": knot_list})


    return Bi, Bg

Bi, Bg = make_basis_splines()
Bg

Next we specify a model maker function to create the various pooled, unpooled and hierarhical (mixed) models of the insurance curve data. Note that even though we're specifying a hierarhical model we have not specified a hierarchy over the insurance codes, instead we have added this as a "fixed" effect feature into our regression model. The idea is that this fixed effect will capture the differences in expected baseline per group code of insurance product. 


In [ ]:
#| output: false

def make_model(loss_df, num_knots=3, max_dev=7, model_type='mixed'):
    Bi, Bg = make_basis_splines(num_knots, max_dev)
    observed = loss_df['lr'].values
    uniques, unique_codes = pd.factorize(loss_df['year_code'])
    coords= {'years': unique_codes, 
            'splines': list(range(Bi.shape[1])) ,
            'measurement': list(range(6)), 
            'obs': uniques
            }

    with pm.Model(coords=coords) as sp_insur:
        basis_g = pm.MutableData('Bg', np.asfortranarray(Bg))

        tau = pm.HalfCauchy('tau', 1)
        ## Global Hierarchical Spline Terms
        beta_g = pm.Normal("beta_g", mu=0, sigma=tau, 
        dims='splines')
        mu_g = pm.Deterministic("mu_g", pm.math.dot(basis_g, beta_g), dims='measurement')

        ## Individual or Year Specific Spline Modifications
        if model_type in ['mixed', 'unpooled']:
            sigma = pm.HalfCauchy('sigma_i', 1)
            basis_i = pm.MutableData('Bi', np.asfortranarray(Bi))
            beta = pm.Normal("beta", mu=0, sigma=sigma, dims=('splines', 'years'))
            mui = pm.Deterministic("mui", pm.math.dot(basis_i, beta), dims=('measurement', 'years'))
        
        ## Features
        prem = pm.MutableData('prem', loss_df['standardised_premium'].values)
        grcode = pm.MutableData('grcode', loss_df['grcode'] == 43)

        beta_prem = pm.Normal('beta_prem', 0, 1)
        beta_grcode = pm.Normal('beta_grcode', 0, 1)
        mu_prem = beta_prem*prem
        mu_grcode = beta_grcode*grcode

        ## Likelihood
        sigma = pm.TruncatedNormal("sigma", 1, lower=0.005)
        if model_type == 'mixed':
            mu = pm.Deterministic('mu',  mu_grcode + mu_prem + (mu_g.T + mui.T).ravel(), dims='obs')
            lr_likelihood = pm.Normal("lr", mu, sigma, observed=observed, dims=('obs'))
        elif model_type == 'pooled': 
             lr_likelihood = pm.Normal("lr",  np.repeat(mu_g, len(unique_codes)), sigma, observed=observed, dims='obs')
        elif model_type == 'unpooled':
            lr_likelihood = pm.Normal("lr",  mui.T.ravel(), sigma, observed=observed, dims=('obs'))


        ## Sampling
        idata_sp_insur = pm.sample(2000, return_inferencedata=True, target_accept=.99,
        idata_kwargs={"log_likelihood": True})
        idata_sp_insur = pm.sample_posterior_predictive(
            idata_sp_insur,extend_inferencedata=True)

    return idata_sp_insur, sp_insur


idata_sp_insur_unpooled, sp_insur_unpooled = make_model(loss_df, model_type='unpooled')
idata_sp_insur_pooled, sp_insur_pooled = make_model(loss_df, model_type='pooled')
idata_sp_insur_mixed, sp_insur_mixed = make_model(loss_df, model_type='mixed')

The model structure can be seen more clearly in this graph. The important features to note are the spline components and the manner of their combination. We are postulating a kind of hierarchy of development curves. We specify the "global" curve in 7 development steps and allow that each distinct year is a realisation of a curve that can be thought of as modifying the profile of "global" curve as `mui` is combined with `mu_g`. These structures are then combined into a more traditional regression and fed into our likelihood term.  


In [ ]:
pm.model_to_graphviz(sp_insur_mixed)


We can extract the effect of the differences grcodes and examine the baseline and annual spline related coefficients. 


In [ ]:
summary = az.summary(idata_sp_insur_mixed, var_names=['sigma', 'beta_grcode', 'beta_prem', 'beta_g', 'beta'])

summary

Again we can compare the performance metrics of the various models. 


In [ ]:
compare_df = az.compare({'unpooled': idata_sp_insur_unpooled, 
            'pooled': idata_sp_insur_pooled, 
            'mixed': idata_sp_insur_mixed})

az.plot_compare(compare_df)
compare_df

The hierarchical splne model is by far the best fit to our data. What does it represent? Yes, splines are function approximation tools. Sure, extrapolation is dangerous, but in this domain where the range of the development curves is constrained they seem to offer an elegant way of articulating the structure of the risk profile.

### Plot the Posterior Predictive Checks

We can check how well the model can recapture the observed data.


In [ ]:
def plot_ppc_splines(idata):
    fig, axs = plt.subplots(2, 5, figsize=(20, 10), sharey=True)
    axs = axs.flatten()
    dev_periods = np.arange(1, 7, 1)
    uniques, unique_codes = pd.factorize(loss_df['year_code'])
    for y, c in zip(unique_codes, range(10)):
        az.plot_hdi(dev_periods, idata['posterior_predictive']['lr'].sel(obs=c), color='firebrick', ax=axs[c], fill_kwargs={'alpha': 0.2}, hdi_prob=.89)
        az.plot_hdi(dev_periods, idata['posterior_predictive']['lr'].sel(obs=c), color='firebrick', ax=axs[c], hdi_prob=0.5)
        axs[c].scatter(dev_periods, loss_df[(loss_df['year_code'] == y)]['lr'], color='k', label='Actual Loss Ratio')
        axs[c].plot(dev_periods, loss_df[(loss_df['year_code'] == y)]['lr'], color='k')
        axs[c].set_title(f"PPC 89% and 50% HDI: {y}")
        axs[c].set_ylabel("Loss Ratio")
        axs[c].legend();

plot_ppc_splines(idata_sp_insur_mixed)

### Plot the Hierarchical Components

In the following plot we show similarly how to recapture the observed data, but additionally we can decompose the structure of the model in each case and extract baseline forecasts which would be our guide to future loss-ratio development curves in lieu of any other information. 


In [ ]:
mu_g = idata_sp_insur_mixed.posterior.stack(draws=("chain", "draw"))["mu_g"]

mu_i = idata_sp_insur_mixed.posterior.stack(draws=("chain", "draw"))["mui"]

mu  = idata_sp_insur_mixed.posterior.stack(draws=("chain", "draw"))['mu']

beta_grcode = idata_sp_insur_mixed.posterior.stack(draws=("chain", "draw"))['beta_grcode']

dev_periods = np.arange(1, 7, 1)
uniques, unique_codes = pd.factorize(loss_df['year_code'])

mosaic = """
         ABCDE
         FGHIJ
         KKKKK
"""
fig, axs = plt.subplot_mosaic(mosaic, sharey=True, 
figsize=(20, 15))
axs = [axs[k] for k in axs.keys()] 

mu_g_mean = mu_g.mean(dim='draws')
for y, c in zip(unique_codes, range(10)):
    group_effect = 0
    if '43' in y: 
        group_effect = beta_grcode.mean().item()
    mu_i_mean = mu_i.sel(years=y).mean(dim='draws')
    axs[c].plot(dev_periods, group_effect + mu_g_mean.values + mu_i_mean.values, label='Combined + E(grp_effect)', color='purple', linewidth=3.5)
    axs[c].plot(dev_periods, group_effect + mu_g_mean.values, label='E(Hierarchical Baseline)', color='red', linestyle='--')
    axs[c].plot(dev_periods,  group_effect + mu_i_mean.values, label='E(Year Specific Adjustment term)', color='blue', linestyle='--')
    axs[c].scatter(dev_periods, loss_df[(loss_df['year_code'] == y)]['lr'], color='k', label='Actual Loss Ratio')
    az.plot_hdi(dev_periods,mu.sel(obs=c).T  , ax=axs[c], color='firebrick', fill_kwargs={'alpha': 0.2})
    az.plot_hdi(dev_periods, mu.sel(obs=c).T , ax=axs[c], color='firebrick', fill_kwargs={'alpha': 0.5}, hdi_prob=.50)
    axs[c].set_title(f"Components for Year {y}")
    axs[c].set_ylabel("Loss Ratio")
    if (c == 0) or (c == 1):
         axs[c].legend()

axs[10].plot(dev_periods, mu_g_mean.values, label='E(Hierarchical Baseline)', color='black')
axs[10].plot(dev_periods, mu_g_mean.values + group_effect, label='E(Hierarchical Baseline) + E(grp_effect)', color='black', linestyle='--')
az.plot_hdi(dev_periods, mu_g.T.values, color='slateblue', ax=axs[10], fill_kwargs={'alpha': 0.2})
az.plot_hdi(dev_periods, mu_g.T.values + group_effect, color='magenta', ax=axs[10], fill_kwargs={'alpha': 0.2})
az.plot_hdi(dev_periods, mu_g.T.values, color='slateblue', ax=axs[10], hdi_prob=.5)
az.plot_hdi(dev_periods, mu_g.T.values  + group_effect, color='magenta', ax=axs[10], hdi_prob=.5)
axs[10].set_title("Baseline Forecast Loss Ratio")
axs[10].legend();


This plot is a bit clunky, because we're mixing expectations and posterior distributions over the parameters. The point is just to highlight the "compositional" structure of our model. A better way to interrogate the implications of the model is to "push" forward different data through the posterior predictive distribution and derive a kind of ceteris paribus rule for accrual of losses.


In [ ]:
with sp_insur_mixed: 
    pm.set_data({'grcode': np.ones(len(loss_df)), 
    })
    idata_43 = pm.sample_posterior_predictive(idata_sp_insur_mixed, var_names=['lr'], extend_inferencedata =True)

with sp_insur_mixed: 
    pm.set_data({'grcode': np.zeros(len(loss_df))})
    idata_353 = pm.sample_posterior_predictive(idata_sp_insur_mixed, var_names=['lr'], extend_inferencedata=True)

idata_353

Even here though we want to average the curves over the specific years in the data and abstract a view of the model implications under different counterfactual settings. Or put another way, we want a view of the average development curve between years, not within a year. If we want to predict the novel insurance loss for next year we need some way to aggregate between the annual trajectories. Here we define a helper function to effect this step. 


In [ ]:
def get_posterior_predictive_curve(idata, prem=2, grcode=1):
    weighted_splines = [np.dot(np.asfortranarray(Bi), az.extract(idata['posterior']['beta'])['beta'].values[:, :, i]) for i in range(2000)]

    weighted_splines_1 = [np.dot(np.asfortranarray(Bg), az.extract(idata['posterior']['beta_g'])['beta_g'].values[:, i]) for i in range(2000)]

    beta_grcode = az.extract(idata['posterior']['beta_grcode'])['beta_grcode']

    beta_prem = az.extract(idata['posterior']['beta_prem'])['beta_prem']
    df1 = pd.DataFrame([beta_prem.values[i]*prem + beta_grcode.values[i]*grcode for i in range(2000)]).T


    ## How do we averaging over the years to get
    ## a view of a new development period?
    #df = pd.concat([pd.DataFrame(weighted_splines_1[i].T + weighted_splines[i].T).mean() for i in range(1000)], axis=1)

    ## Sample random group each draw from posterior
    df = pd.concat([pd.DataFrame((weighted_splines_1[i].T + weighted_splines[i].T)[np.random.choice(list(range(10))), :]) for i in range(2000)], axis=1)

    ## Average random subset of of groups
    #df = pd.concat([pd.DataFrame((weighted_splines_1[i].T + weighted_splines[i].T)[np.random.choice(list(range(10)), 5), :]).mean() for i in range(2000)], axis=1)

    df = df1.iloc[0].values + df

    return df

pred_df_1 = get_posterior_predictive_curve(idata_43, prem=1, grcode=1)
pred_df_0 = get_posterior_predictive_curve(idata_353, prem=1, grcode=0) 

fig, ax = plt.subplots(figsize=(9, 7), sharey=True)

az.plot_hdi(range(6), pred_df_0.values.T, ax=ax, color='slateblue', smooth=False, fill_kwargs={'alpha': 0.2})
az.plot_hdi(range(6), pred_df_1.values.T, ax=ax, color='firebrick', smooth=False, fill_kwargs={'alpha': 0.2})
az.plot_hdi(range(6), pred_df_0.values.T, ax=ax, color='slateblue', smooth=False, fill_kwargs={'alpha': 0.5}, hdi_prob=.5)
az.plot_hdi(range(6), pred_df_1.values.T, ax=ax, color='firebrick', smooth=False, fill_kwargs={'alpha': 0.5}, hdi_prob=.5)
ax.plot(pred_df_0.mean(axis=1), linestyle='-', color='k', linewidth=4, label='grcode 353 prem 1')

ax.plot(pred_df_1.mean(axis=1), linestyle='--', color='grey', linewidth=4, label='grcode 43 prem 1')

ax.set_title("Posterior Samples of the Trajectories \n Under different Counterfactual settings")
ax.set_ylabel("Loss Ratio")
ax.set_xlabel("Development Period")
ax.legend();

Note how we've varied the preditor inputs to showcase the expected realisation under different product codes. 

## Multiple Smoother Regression Models

To demonstrate how spline modelling can be further adapted to the multiple regression like cases we use the PISA data set discussed in [this case study](https://m-clark.github.io/generalized-additive-models/application.html) from Michael Clark. 

The data set has been constructed using average Science scores by country from the Programme for International Student Assessment (PISA) 2006, along with GNI per capita (Purchasing Power Parity, 2005 dollars), Educational Index, Health Index, and Human Development Index from UN data. We want to model the overall outcome score as a function of these broad demographic features. 


In [ ]:
pisa_df = pd.read_csv("https://raw.githubusercontent.com/m-clark/generalized-additive-models/master/data/pisasci2006.csv")

pisa_df.head()


The relationships displayed between each of these measures is not obviously linear, and as such could plausibly benefit from being modelled with splines. 


In [ ]:
g = sns.pairplot(pisa_df[['Overall', 'Income', 'Support', 'Health', 'Edu']],  kind="reg", height=2)
g.fig.suptitle("Pair Plot of Complex Relations", y=1.05);

We define three models for contrasting the implications. Note here how we have to define a seperate spline basis for each of the covariates. Here we create the knots for defining our basis on each covariate. 


In [ ]:
knots_income = np.linspace(np.min(pisa_df['Income']), np.max(pisa_df['Income']), 5+2)[1:-1]

knots_edu = np.linspace(np.min(pisa_df['Edu']), np.max(pisa_df['Edu']), 5+2)[1:-1]

knots_health = np.linspace(np.min(pisa_df['Health']), np.max(pisa_df['Health']), 5+2)[1:-1]

knots_income1 = np.linspace(np.min(pisa_df['Income']), np.max(pisa_df['Income']), 3+2)[1:-1]

knots_edu1 = np.linspace(np.min(pisa_df['Edu']), np.max(pisa_df['Edu']), 3+2)[1:-1]

knots_health1 = np.linspace(np.min(pisa_df['Health']), np.max(pisa_df['Health']), 3+2)[1:-1]

Now we initialise these models


In [ ]:
formula = "Overall ~ Income + Edu + Health"
base_model = bmb.Model(formula, pisa_df, dropna=True)

formula_spline = """Overall ~ bs(Income, degree=3, knots=knots_income) + bs(Edu, degree=3, knots=knots_edu) + bs(Health, degree=3, knots=knots_health) """

formula_spline1 = """Overall ~ (1 | Country) + bs(Income, degree=3, knots=knots_income1) + bs(Edu, degree=3, knots=knots_edu1) + bs(Health, degree=3, knots=knots_health1) """

spline_model = bmb.Model(formula_spline, pisa_df, dropna=True)

spline_model1 = bmb.Model(formula_spline1, pisa_df, dropna=True)

spline_model1

In [ ]:
spline_model.build()
spline_model.graph()

In [ ]:
#| output: false
#| 
base_idata = base_model.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True})
spline_idata = spline_model.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True}, target_accept=.95)
spline_idata1 = spline_model1.fit(random_seed=random_seed, idata_kwargs={"log_likelihood": True}, target_accept=.99)

We can compare the simple regression approach to the spline based regression in the now usual way. 


In [ ]:
compare_df = az.compare({'spline': spline_idata, 'raw': base_idata, 'spline_hierarchy':spline_idata1})

compare_df


The coefficients comparisons are harder


In [ ]:
az.summary(base_idata)

since it's less clear what the spline coefficient terms mean. 


In [ ]:
az.summary(spline_idata)

We can also check that the model seems to recover the observed data well. 

### Plotting Posterior Predictive Checks


In [ ]:
base_model.predict(base_idata, kind='pps')
spline_model.predict(spline_idata, kind='pps')
spline_model1.predict(spline_idata1, kind='pps')


fig, axs = plt.subplots(1, 3, figsize=(20, 8))
axs = axs.flatten()
az.plot_ppc(base_idata, ax=axs[0])
az.plot_ppc(spline_idata, ax=axs[1]);
az.plot_ppc(spline_idata1, ax=axs[2]);
axs[0].set_xlabel('')
axs[1].set_title("PPC: Spline Model");
axs[0].set_title("PPC: Regression Model");
axs[2].set_title("PPC: Hierarchical Spline Model");

Next we'll dig into the spline basis features and decompose the predicted outcome and show how the outcome varies for levels in the inputs variables. The Bambi interpret package offers some functionality to assess the conditional predictions for varying values of the input variables. 

### Model Interpretability Plots in Bambi

In particular we want to highlight the differences between the hierarchical and non-hierarchical multivariable spline models. The shrinkage effects of hierarchically modelling country intercepts are clearly evident and constrain the poor extrapolation effects of the simpler spline model.


In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(9, 14))
axs = axs.flatten()
bmb.interpret.plot_predictions(
    spline_model, spline_idata, "Income", ax=axs[0]
)

axs[0].set_title("Non-Hierarchical Income")
axs[2].set_title("Non-Hierarchical Edu")
axs[4].set_title("Non-Hierarchical Health")

axs[1].set_title("Hierarchical Income")
axs[3].set_title("Hierarchical Edu")
axs[5].set_title("Hierarchical Health")


bmb.interpret.plot_predictions(
    spline_model, spline_idata, "Edu", ax=axs[2]
)

bmb.interpret.plot_predictions(
    spline_model, spline_idata, "Health", ax=axs[4]
);

bmb.interpret.plot_predictions(
    spline_model1, spline_idata1, "Income", 
    sample_new_groups=True, ax=axs[1]
)

bmb.interpret.plot_predictions(
    spline_model1, spline_idata1, "Edu", sample_new_groups=True, ax=axs[3]
)

bmb.interpret.plot_predictions(
    spline_model1, spline_idata1, "Health",sample_new_groups=True, ax=axs[5]
);


We can pull these types of values out into a table. Note here how we ask for predictions based on varying values of the `Edu` and `Income` variables where we keep the `Health` variable fixed at the mean value. 


In [ ]:
summary_df = bmb.interpret.predictions(
    spline_model,
    spline_idata,
    conditional={
        "Edu": list(np.linspace(0.5, .98, 10)),
        "Income": list(np.linspace(0.4, .98, 10)),
        },
)
summary_df


summary_df1 = bmb.interpret.predictions(
    spline_model1,
    spline_idata1,
    conditional={
        "Edu": list(np.linspace(0.5, .98, 10)),
        "Income": list(np.linspace(0.4, .98, 10)),
        },
    sample_new_groups=True
)
summary_df1


We can then plot these results based on the conditional realisations and see some interesting behaviour at the implausilble reaslisations. This should make us somewhat wary of using this model to extrapolate too far beyond the observable range of data


In [ ]:
g = sns.relplot(data=summary_df, x="Income", y="estimate", hue="Edu")

g.fig.suptitle("Marginal Predictions of the Outcome Variable \n conditional on counterfactual values for Edu and Income", y=1.05);


g = sns.relplot(data=summary_df1, x="Income", y="estimate", hue="Edu")

g.fig.suptitle("Hierarchical Marginal Predictions of the Outcome Variable \n conditional on counterfactual values for Edu and Income", y=1.05);

However, we can see here how the hierarchical component shrinks the predicted values back towards a reasonable range! Constraining the poor extrapolation of the more naive spline model. This demonstrates something of the interplay between in-sample approximation and out of sample generalisation we saw above in the case of the insurance curve development. The additional structure insists on commensurate realisations under different counterfactual settings reflective of the fact that countries do not vary so radically. 

### The Spline Component Contributions

Finally, we'll pull out the component contributions of each variable and see how they combine additively. This will also serve as a kind of posterior predictive check for each country as we can show the degree which posterior draws from each component sum to achieve a plausible mirror of the observed data.  


In [ ]:
Bincome = spline_model.response_component.design.common['bs(Income, degree=3, knots=knots_income)']

income_coefs = az.extract(spline_idata['posterior']['bs(Income, degree=3, knots=knots_income)'])['bs(Income, degree=3, knots=knots_income)']

Bedu = spline_model.response_component.design.common['bs(Edu, degree=3, knots=knots_edu)']

edu_coefs = az.extract(spline_idata['posterior']['bs(Edu, degree=3, knots=knots_edu)'])['bs(Edu, degree=3, knots=knots_edu)']


Bhealth = spline_model.response_component.design.common['bs(Health, degree=3, knots=knots_health)']

health_coefs = az.extract(spline_idata['posterior']['bs(Health, degree=3, knots=knots_health)'])['bs(Health, degree=3, knots=knots_health)']

income = np.dot(Bincome, income_coefs).T 
edu = np.dot(Bedu, edu_coefs).T
health = np.dot(Bhealth, health_coefs).T

intercept = az.extract(spline_idata['posterior']['Intercept'])['Intercept'].values

fig, ax = plt.subplots(figsize=(10, 7))
for i in range(100):
    if i == 1:
        ax.plot(income[i], label='Income Component', color='red')
        ax.plot(edu[i], label='Edu Component', color='blue')
        ax.plot(health[i], label='Health Component', color='darkgreen')
        ax.plot(intercept[i] + income[i] + edu[i] + health[i], label='Combined Components', color='purple')
    else: 
        ax.plot(income[i], alpha=0.1, color='red')
        ax.plot(edu[i], alpha=0.1, color='blue')
        ax.plot(health[i], alpha=0.1, color='darkgreen')
        ax.plot(intercept[i] + income[i] + edu[i] + health[i], color='purple', alpha=0.3)

ax.scatter(range(len(spline_idata['observed_data']['Overall'])), spline_idata['observed_data']['Overall'], label='Observed', color='grey', s=56, ec='black')
ax.set_title("Additive Spline Components", fontsize=20)
ax.legend();
ax.set_xticklabels(pisa_df.dropna(axis=0).reset_index()['Country']);

We can see here how the combined components borrow the structure of the outcome variable primarily from the income variable component. The health measures have closer to zero additive contribution while the uncertainty in the educational component varies wildly. But the blue educational component here is used primarily as a scaling contribution which adds to the level of the outcome variable rather than distorting the shape. These initially opaque synthetic features of splines can offer deep insight into the structure of our data generating process when seen correctly. 

## Conclusion

We've seen the application of splines as univariate smoothers to approximate wiggly curves of arbitrary shape. We've also tried gaussian process approximations of the same univariate functions. The suggested flexibility of both methods is a strength, but we need to be careful where splines have a tendency to over-fit to individual curves. As such we have tried to show that we can incorporate spline basis modelling in a hierarchical bayesian model and recover more compelling posterior predictive checks and additionally derive predictiions from the mixed variant of the hierarhical model which helps us understand the implications of the data for generic forecasts of insurance loss curves. Finally we showed how splines can be used additively to model non-linear functions of multiple covariates. These are a powerful tool to interrogate complex non-linear relationships, but they offer interpolation of functions within a well understood domain. Applealing to these models for extrapolation needs to be done carefully. 

We can, I think, draw a broad moral from this presentation. Observed phenomena admit description with a wide variety of mathematical abstraction. The "correctness" of the abstraction is never the right question. The manner in which one abstraction distils the structure of reality is not fundamentally any better than the manner in which an alternative scheme will do so. Predictive success achieved applying one abstraction can be traded off for interpretive tractability in another. What matters is the task at hand - solving the problem. Sometimes problems can be solved with new information, but mostly they're resolved by re-arranging assumptions and phrasing the problem anew - seeing through a different lens, pulling the door handle where we previously pushed. Splines, hierarhical splines, gaussian process modelling are all attempts at approximating an answer by rephrasing the question. The more of these tools we have the better we are prepared to tackle novel problems, discern the pertinent structure in the world. 